In [3]:

library(readr)
library(stringr)
library (ggplot2)
library(dplyr)
library(forcats)
library(gridExtra)


In [9]:

library(haven)
vndn_hoz <- read_sav("HHOLD.sav")

vndn_ind <- read_sav("IND.sav")



In [10]:
##Почистим данные, и оставим только необходимые столбцы

vndn2018_hoz = vndn_hoz %>% select(R_H_DOXOD_TR, R_H_DR_DEN, R_H_SOBST, R_H04_14_CHIST)

##Сначала посчитаем сумму общего рыночного дохода по домохозяйствам
###R_H_DOXOD_TRТ1_I Доход от трудовой деятельности – всего
###R_H_DR_DENТ1_3.3 Другие денежные поступления от частных лиц и организаций, помимо органов социальной защиты населения
###R_H_SOBSTК3_4 (Т1_7.3) Доходы от собственности
###R_H04_14_CHISTК3_5 (Т1_6.3.1) (Т1_7.2) Чистый доход от проживания в собственном жилье (чистая вмененная арендная плата), год

#Сначала посчитаем по строкам, потом общую

vndn2018_hoz$sum <- rowSums(vndn2018_hoz, na.rm = TRUE, dims = 1)

sum(vndn2018_hoz$sum)

##27,721,072,921

##А теперь посчитаем сумму общего рыночного дохода по лицам

###I03_44 Сумма дохода от подработки за год
###R_I_SAM К1_2 (Т1_1.2.1) Расчетная сумма дохода от предпринимательской деятельности до выплаты налогов, по месту основной работы (лицо 16 лет и более) (самозанятость)
###R_I_DOP К1_3 (Т1_1.3.1) Общая сумма заработка (дохода) за выполнение дополнительной работы (наряду с основной) до выплаты подоходного налога – всего


vndn2018_ind = vndn_ind %>% select(I03_44, R_I_SAM, R_I_DOP)

vndn2018_ind$sum <- rowSums(vndn2018_ind, na.rm = TRUE, dims = 1)

sum(vndn2018_ind$sum)

##1,280,684,461

##Проделаем все то же самое с совокупным дохом (total income)

###R_H_DOXOD_TRТ1_I Доход от трудовой деятельности – всего
###R_H_DR_DENТ1_3.3 Другие денежные поступления от частных лиц и организаций, помимо органов социальной защиты населения
###R_H_SOBSTК3_4 (Т1_7.3) Доходы от собственности
###R_H04_14_CHISTК3_5 (Т1_6.3.1) (Т1_7.2) Чистый доход от проживания в собственном жилье (чистая вмененная арендная плата), год
###+ R_H_DOX_SOV Т1_VII Совокупный доход - всего (включая оценочный эквивалент чистой вмененной арендной платы) Сумма денежного дохода и поступлений в натуральной форме (в денежном выражении) Включает в себя: - доходы от трудовой деятельности (включая натуральные поступления) - чистый доход от проживания в собственном жилье (оценочный эквивалент чистой вмененной арендной платы) - доходы от собственности - трансферты – полученные в денежной и натуральной форме

#Сначала по домохозяйствам

vndn2018_hoz_total = vndn_hoz %>% select(R_H_DOXOD_TR, R_H_DR_DEN, R_H_SOBST, R_H04_14_CHIST, R_H_DOX_SOVK)

vndn2018_hoz_total$sum <- rowSums(vndn2018_hoz_total, na.rm = TRUE, dims = 1)

sum(vndn2018_hoz_total$sum)

##66,892,918,680


##Потом по индивидуальным лицам
### Сумма дохода от подработки за год
###R_I_SAM К1_2 (Т1_1.2.1) Расчетная сумма дохода от предпринимательской деятельности до выплаты налогов, по месту основной работы (лицо 16 лет и более) (самозанятость)
###R_I_DOP К1_3 (Т1_1.3.1) Общая сумма заработка (дохода) за выполнение дополнительной работы (наряду с основной) до выплаты подоходного налога – всего
###R_I_PENS  К2_1 (Т1_3.1.1) Пенсии, назначенные лицам до 16 лет и лицам 16 лет и более


vndn2018_ind_total = vndn_ind %>% select(I03_44, R_I_SAM, R_I_DOP, R_I_PENS)

vndn2018_ind_total$sum <- rowSums(vndn2018_ind_total, na.rm = TRUE, dims = 1)

sum(vndn2018_ind_total$sum)


##9,481,286,178


#Для удобства переименуем


names(vndn2018_hoz)[names(vndn2018_hoz) == 'sum'] <- 'hhold_market_income'

names(vndn2018_ind)[names(vndn2018_ind) == 'sum'] <- 'ind_market_income'

names(vndn2018_hoz_total)[names(vndn2018_hoz_total) == 'sum'] <- 'hhold_total_income'

names(vndn2018_ind_total)[names(vndn2018_ind_total) == 'sum'] <- 'ind_total_income'


#Создаем один датасет из всех предыдущих


vndn2018 <- bind_rows(vndn2018_hoz, vndn2018_ind, vndn2018_hoz_total, vndn2018_ind_total)


#Выбираем только необходимые колонки

vndn2018 = vndn2018 %>% select(hhold_market_income, ind_market_income,hhold_total_income,ind_total_income)


#Наконец считаем сумму по стоkбцам и находим необходимые значения по всем 4 метрикам


colSums(vndn2018, na.rm = TRUE, dims = 1)

##hold_market_income   ind_market_income  hhold_total_income    ind_total_income 
#        27721072921          1280684461         66892918680          9481286178 


##Сравним показатели с данными росстата -- возникла проблема. 



[1] 27721072921

[1] 1280684461

[1] 66892918680

[1] 9481286178

Warning message in bind_rows_(x, .id):
“Vectorizing 'haven_labelled' elements may not preserve their attributes”
Warning message in bind_rows_(x, .id):
“Vectorizing 'haven_labelled' elements may not preserve their attributes”


hhold_market_income   ind_market_income  hhold_total_income    ind_total_income 
        27721072921          1280684461         66892918680          9481286178